# Formação Cientista de Dados - Data Science Academy
### Prevendo Customer Churn em Operadoras de Telecom
Projeto com Feedback 4

In [ ]:
import re
import pandas as pd
import numpy as np
import seaborn as sns
import statsmodels.api as sm
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import chi2
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, f1_score, recall_score
from sklearn.ensemble import RandomForestClassifier
from statsmodels.stats.outliers_influence import variance_inflation_factor

### Funções Utilitarias

In [ ]:
# Carregar dados
def carrega_dados(nome_arquivo):
    return pd.read_csv(nome_arquivo)

In [ ]:
# Gerar gráficos de balanceamento de classes
def plot_balanceamento_classes(classe, titulo):
    classes, counts = np.unique(classe, return_counts=True)
    plt.bar(classes, counts)
    plt.xticks(classes)
    plt.xlabel('Classes')
    plt.ylabel('Número de amostras')
    plt.title(titulo)
    
    for i in range(len(classes)):
        plt.text(x = classes[i] - 0.05, y = counts[i] + 1, s = counts[i])
    
    plt.show()

In [ ]:
# Aplicar de label-encoding
def aplica_label_encoding(df, lista_colunas):
    le = LabelEncoder()
    for coluna in lista_colunas:
        df[coluna] = le.fit_transform(df[coluna])
    return df

In [ ]:
# Selecionando melhores variaveis para o modelo
def selecionar_melhores_variaveis(x, y, quatidade_variaveis):
    logreg = LogisticRegression(max_iter=1200)
    rfe = RFE(estimator=logreg, n_features_to_select=quatidade_variaveis)
    rfe.fit(x, y)
    return x.columns[rfe.support_]

In [ ]:
# Validando se possui/não possui informações
def validar_valores(valor):
    if valor > 0:
        return 1
    else:
        return 0

In [ ]:
# Aplicando o SMOTE para balancear as classes
def aplicar_balanceamento_classes(variavel_alvo, variaveis_preditoras):
    smote = SMOTE(random_state=42)
    return smote.fit_resample(variavel_alvo, variaveis_preditoras)

### Análise exploratória de dados

In [ ]:
# Carregando dados de Treino
df_treino = carrega_dados("datasets/train.csv")

print('dimensões:', df_treino.shape)
print('colunas:', df_treino.columns)

In [ ]:
# Analisando dataframe
df_treino.head(5)

In [ ]:
# Selecionando coluna de indice
colunas_deletar = ['Unnamed: 0']

# Selecionando colunas com valores semelhantes
#colunas_deletar = colunas_deletar + ['area_code']
print('colunas que serão removidas:', colunas_deletar)

In [ ]:
# Analisando dados que serão excluidos
df_colunas_deletar = df_treino[colunas_deletar]
df_colunas_deletar.head(10)

In [ ]:
# Removendo colunas
df_modificado = df_treino.drop(colunas_deletar, axis=1)

In [ ]:
# Analisando dataframe após remover as colunas
print('dimensões:', df_modificado.shape)
print('tipos das variáveis:\n', df_modificado.dtypes)

df_modificado.head(5)

In [ ]:
# Definir as categorias que serão usadas no gráfico
categories = ['Não Churn', 'Churn']

# Calcular a média de chamadas de serviço ao cliente para cada valor de churn
churn_calls = df_modificado.groupby('churn')['number_customer_service_calls'].mean()

# Criar um gráfico de barras para mostrar as médias de chamadas de serviço ao cliente para cada valor de churn
plt.bar(categories, churn_calls.values)
plt.title('Média de chamadas de serviço ao cliente por churn')
plt.xlabel('Churn')
plt.ylabel('Média de chamadas de serviço ao cliente')

# Adicionar os valores das barras no gráfico
for i, value in enumerate(churn_calls.values):
    plt.text(i, value, str(round(value, 2)), ha='center')

plt.show()

In [ ]:
churn_by_area = df_modificado.groupby('area_code')['churn'].value_counts()
print(churn_by_area)

In [ ]:
max_count = churn_by_area.max()
max_area_codes = churn_by_area[churn_by_area == max_count].index.get_level_values(0)
print("Área(s) com o maior número de churns:", max_area_codes)

In [ ]:
# Criar uma tabela pivô com a contagem de "yes" e "no" por "area_code"
df_pivot = df_modificado.pivot_table(index='area_code', columns='churn', values='state', aggfunc='count')

# Transformar a tabela pivô em um DataFrame com as colunas "area_code", "churn" e "count"
df_new = pd.DataFrame(df_pivot.to_records())
df_new = df_new.rename(columns={'area_code': 'Area Code', 'no': 'No', 'yes': 'Yes'})

# Plotar um gráfico de barras com as colunas "yes" e "no" lado a lado
ax = df_new.plot(x='Area Code', kind='bar', rot=0)

# Adicionar rótulos de valores às barras
for c in ax.containers:
    for r in c:
        h = r.get_height()
        ax.annotate(f'{h:.0f}', xy=(r.get_x() + r.get_width() / 2, h), 
                    xytext=(0, 3), textcoords="offset points", ha='center', va='bottom')

# Adicionar rótulos de rótulos à base das barras
ax.set_xlabel('Area Code')
ax.set_ylabel('Count')
ax.set_title('Churn Count by Area Code')

# Mostrar o gráfico
plt.show()

In [70]:
# Analisando área com maio número de cancelamentos
churn_by_area = df_modificado.groupby('area_code')['churn'].value_counts()
max_count = churn_by_area.max()
max_area_codes = churn_by_area[churn_by_area == max_count].index.get_level_values(0)
max_area_codes

df_maior_valor_churn = df_modificado.loc[(df_modificado['area_code'] == 'area_code_408') & (df_modificado['churn'] == 'yes')]
print('dimenssões:', df_maior_valor_churn.shape)
df_maior_valor_churn.head(5)

dimenssões: (122, 20)


,state,account_length,area_code,international_plan,voice_mail_plan,number_vmail_messages,total_day_minutes,total_day_calls,total_day_charge,total_eve_minutes,total_eve_calls,total_eve_charge,total_night_minutes,total_night_calls,total_night_charge,total_intl_minutes,total_intl_calls,total_intl_charge,number_customer_service_calls,churn
21,CO,77,area_code_408,no,no,0,62.4,89,10.61,169.9,121,14.44,209.6,64,9.43,5.7,6,1.54,5,yes
33,AZ,12,area_code_408,no,no,0,249.6,118,42.43,252.4,119,21.45,280.2,90,12.61,11.8,3,3.19,1,yes
41,MD,135,area_code_408,yes,yes,41,173.1,85,29.43,203.9,107,17.33,122.2,78,5.50,14.6,15,3.94,0,yes
57,CO,121,area_code_408,no,yes,30,198.4,129,33.73,75.3,77,6.40,181.2,77,8.15,5.8,3,1.57,3,yes
77,NY,144,area_code_408,no,no,0,61.6,117,10.47,77.1,85,6.55,173.0,99,7.79,8.2,7,2.21,4,yes


In [ ]:
# Verificar se precisa tratar outlier da variavel number_vmail_messages

# Analisando variáveis que podem ser modificadas por sim/não
print('valores de number_vmail_messages:', df_modificado['number_vmail_messages'].unique())
print('valores de number_customer_service_calls:', df_modificado['number_customer_service_calls'].unique())
df_modificado.sample(5)

In [ ]:
# Está tecnica aplicada apresentou piora na performance do modelo 
# Modificando colunas de valores numericos para binários
# df_modificado['number_vmail_messages'] = df_modificado['number_vmail_messages'].apply(validar_valores)
# df_modificado['number_customer_service_calls'] = df_modificado['number_customer_service_calls'].apply(validar_valores)
# df_modificado.sample(5)

In [ ]:
# Analisando distribuição dos dados
df_modificado.hist(bins=20, figsize=(10,10))
plt.show()

### Tratamento dos dados

In [ ]:
# Selecionando variáveis categóricas
variaveis_categoricas = df_modificado.select_dtypes(include='object')

In [ ]:
# Analisando valores únicos das variáveis categóricas
for v in variaveis_categoricas:
    print('valores únicos da variável', v, ':', list(df_modificado[v].unique()))

print('\ncolunas modificadas:', list(variaveis_categoricas.columns))

In [ ]:
# Aplicando label-encoding nas variáveis categóricas
df_modificado = aplica_label_encoding(df_modificado, variaveis_categoricas.columns)
df_modificado.head(5)

In [ ]:
# Analisando correlação das variáveis preditoras com a variável alvo

# Separando as variáveis preditoras e alvo
variavel_alvo = df_modificado['churn']
variaveis_preditoras = df_modificado.drop('churn', axis=1)

# Criando a matriz de correlação
matriz_correlacao_alvo = variaveis_preditoras.corrwith(variavel_alvo).sort_values(ascending=False)
matriz_correlacao_alvo = pd.DataFrame({'variavel': matriz_correlacao_alvo.index, 'correlacao': matriz_correlacao_alvo.values})

print(matriz_correlacao_alvo)

In [ ]:
# Analisando correlação das variáveis

# Temos 2 possibilidade de remoção, definindo uma quantidade ou os numeros iniciais de cada correlação
#colunas_baixa_correlacao = matriz_correlacao_alvo.iloc[(matriz_correlacao_alvo['correlacao'] - 0).abs().argsort()[:4]]
colunas_baixa_correlacao = list(matriz_correlacao_alvo[matriz_correlacao_alvo["correlacao"].astype(str).str.startswith("0.00")]['variavel'])
print('colunas com baixa correlação:', colunas_baixa_correlacao)

In [ ]:
# Removendo variáveis com baixo nível correlação
# Não é necessário remover
#df = df_modificado.drop(colunas_baixa_correlacao, axis=1)
df = df_modificado
print('\ndimensões:', df.shape)
df.head(3)

In [ ]:
# Aplicando análise de Variance inflation factor (VIF)
# Selecionando variável alvo
variaveis_preditoras = df.drop('churn', axis=1)

# Criando constante
X = sm.add_constant(variaveis_preditoras)

# Criando modelo
model = sm.OLS(df['churn'], X)

# Treinando o modelo
results = model.fit()

# Calculando o VIF para cada variável
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif["features"] = X.columns
vif

#  Selecionando variáveis com alto indice VIF
colunas_alto_vif = vif[(vif["features"] != "const") & (vif["VIF Factor"] > 5)]
colunas_alto_vif = colunas_alto_vif["features"].values
colunas_alto_vif

# Removendo variáveis com alto indice VIF
print('Após testes realizados, constatei que tivemos um aumento de mais de 3% de acurácia não removendo as variáveis.')
#df = df.drop(colunas_alto_vif, axis=1)
#print('dimensões:', df.shape)
#df.head(3)

In [ ]:
# Analisando a distribuição das classes
plot_balanceamento_classes(df['churn'], 'Distribuição das classes no DataFrame original')

In [ ]:
# Separando varáveis preditoras e alvo
variavel_alvo = df.drop('churn', axis=1)
variaveis_preditoras = df['churn']

In [ ]:
# Aplicando balanceamento de classes
x_balanceado, y_balanceado = aplicar_balanceamento_classes(variavel_alvo, variaveis_preditoras)

In [ ]:
# Analisando a distribuição das classes após balanceamento
plot_balanceamento_classes(y_balanceado, 'Distribuição das classes após a aplicação do SMOTE')

In [ ]:
x_balanceado.sample(5)

In [ ]:
# Selecionando variáveis numéricas
lista_colunas_nao_numericas = ['international_plan', 'voice_mail_plan']

colunas_numericas = x_balanceado.drop(lista_colunas_nao_numericas, axis=1)
print('lista_colunas_numericas:', colunas_numericas.columns)

# Criando o StandardScaler para aplicar nas variáveis
scaler = StandardScaler()

In [ ]:
# Analisando valores que serão normalizados.
t = x_balanceado[colunas_numericas.columns].head(5)

In [ ]:
# Aplicando padronização
x_balanceado[colunas_numericas.columns] = scaler.fit_transform(colunas_numericas)

In [ ]:
x_balanceado.head(5)

In [ ]:
# Selecionando as melhores variáveis para o modelo
melhores_variaveis = selecionar_melhores_variaveis(x_balanceado, y_balanceado, 11)

# Exibindo as variáveis selecionadas
print("Variáveis selecionadas:", melhores_variaveis)

In [ ]:
# Analisando dados atuais
x_balanceado.sample(10)

In [ ]:
# Analisando dados somente com as variáveis selecionadas
x_balanceado = x_balanceado[melhores_variaveis]
x_balanceado.head(5)

### Modelo de Regressão Logística

In [ ]:
# Dividindo o conjunto de dados em treino e teste
x_treino, x_teste, y_treino, y_teste = train_test_split(x_balanceado, y_balanceado, test_size=0.3, random_state=42)

In [ ]:
# Criando o modelo
model = LogisticRegression()

In [ ]:
# Trinando o modelo
model.fit(x_treino, y_treino)

In [ ]:
# Fazendo previsões
y_pred = model.predict(x_teste)

In [ ]:
# Avaliando o modelo
accuracy = accuracy_score(y_teste, y_pred)
recall = recall_score(y_teste, y_pred)
f1 = f1_score(y_teste, y_pred)
auc_roc = roc_auc_score(y_teste, y_pred)

In [ ]:
print("Acurácia: {:.2f}%".format(accuracy * 100))
print("Recall: {:.2f}%".format(recall * 100))
print("F1-score: {:.2f}%".format(f1 * 100))
print("AUC-ROC: {:.2f}%".format(auc_roc * 100))